In [20]:
import sys

arguments = len(sys.argv)-1
arguments

2

In [18]:
sys.argv

['/home/dylan/.local/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/dylan/.local/share/jupyter/runtime/kernel-2d4c2103-ca31-45b1-8fc7-f19aec3d75db.json']

In [1]:
import numpy as np
import xgcm
from xgcm import Grid
from xhistogram.xarray import histogram
import xarray as xr
import xroms
from scipy import signal
import cmocean.cm as cmo
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import traceback
# from dask.distributed import Client
# client = Client()

In [7]:
try:
    ds = xroms.open_netcdf('/d1/shared/shelf_ho_0_dh_0_vwind_0_his.nc', 
                         chunks = {'ocean_time':1})
    ds, grid = xroms.roms_dataset(ds, 
                                  Vtransform = None)
except AttributeError:
    ds = xr.open_dataset('/d1/shared/shelf_ho_0_dh_0_vwind_0_his.nc', 
                         chunks = {'ocean_time':1})
    coords={'xi':{'center':'xi_rho', 'inner':'xi_u'}, 
        'eta':{'center':'eta_rho', 'inner':'eta_v'}, 
        's':{'center':'s_rho', 'outer':'s_w'}}

    grid = xgcm.Grid(ds, coords=coords, periodic=[])

    Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho
    Zo_w = (ds.hc * ds.s_w + ds.Cs_w * ds.h) / (ds.hc + ds.h)
    z_w = ds.zeta + (ds.zeta + ds.h) * Zo_w

    ds.coords['z_rho'] = z_rho.transpose('ocean_time', 's_rho', 'eta_rho', 'xi_rho')
    ds.coords['z_w'] = z_w.transpose('ocean_time', 's_w', 'eta_rho', 'xi_rho')

In [12]:
ds

/home/dylan/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


<xarray.Dataset>
Dimensions:         (Nuser: 25, eta_psi: 129, eta_rho: 130, eta_u: 130, eta_v: 129, ocean_time: 481, s_rho: 30, s_w: 31, tracer: 3, xi_psi: 261, xi_rho: 262, xi_u: 261, xi_v: 262)
Coordinates:
  * s_rho           (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w             (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.06667 -0.03333 0.0
    x_rho           (eta_rho, xi_rho) float64 dask.array<chunksize=(130, 262), meta=np.ndarray>
    y_rho           (eta_rho, xi_rho) float64 dask.array<chunksize=(130, 262), meta=np.ndarray>
    x_u             (eta_u, xi_u) float64 dask.array<chunksize=(130, 261), meta=np.ndarray>
    y_u             (eta_u, xi_u) float64 dask.array<chunksize=(130, 261), meta=np.ndarray>
    x_v             (eta_v, xi_v) float64 dask.array<chunksize=(129, 262), meta=np.ndarray>
    y_v             (eta_v, xi_v) float64 dask.array<chunksize=(129, 262), meta=np.ndarray>
  * ocean_time      (ocean_time) object 0001-01-01 00:00:00 ... 0001-03-02 00...
    z_rho           (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 30, 130, 262), meta=np.ndarray>
    z_w             (ocean_time, s_w, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 31, 130, 262), meta=np.ndarray>
Dimensions without coordinates: Nuser, eta_psi, eta_rho, eta_u, eta_v, tracer, xi_psi, xi_rho, xi_u, xi_v
Data variables:
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          object ...
    nHIS            int32 ...
    ndefHIS         int32 ...
    nRST            int32 ...
    ntsAVG          int32 ...
    nAVG            int32 ...
    ndefAVG         int32 ...
    ntsDIA          int32 ...
    nDIA            int32 ...
    ndefDIA         int32 ...
    Falpha          float64 ...
    Fbeta           float64 ...
    Fgamma          float64 ...
    Akt_bak         (tracer) float64 dask.array<chunksize=(3,), meta=np.ndarray>
    Akv_bak         float64 ...
    Akk_bak         float64 ...
    Akp_bak         float64 ...
    rdrg            float64 ...
    rdrg2           float64 ...
    Zob             float64 ...
    Zos             float64 ...
    gls_p           float64 ...
    gls_m           float64 ...
    gls_n           float64 ...
    gls_cmu0        float64 ...
    gls_c1          float64 ...
    gls_c2          float64 ...
    gls_c3m         float64 ...
    gls_c3p         float64 ...
    gls_sigk        float64 ...
    gls_sigp        float64 ...
    gls_Kmin        float64 ...
    gls_Pmin        float64 ...
    Charnok_alpha   float64 ...
    Zos_hsig_alpha  float64 ...
    sz_alpha        float64 ...
    CrgBan_cw       float64 ...
    Znudg           float64 ...
    M2nudg          float64 ...
    M3nudg          float64 ...
    Tnudg           (tracer) float64 dask.array<chunksize=(3,), meta=np.ndarray>
    Tnudg_SSS       float64 ...
    rho0            float64 ...
    R0              float64 ...
    Tcoef           float64 ...
    Scoef           float64 ...
    gamma2          float64 ...
    LuvSrc          int32 ...
    LwSrc           int32 ...
    LtracerSrc      (tracer) int32 dask.array<chunksize=(3,), meta=np.ndarray>
    LsshCLM         int32 ...
    Lm2CLM          int32 ...
    Lm3CLM          int32 ...
    LtracerCLM      (tracer) int32 dask.array<chunksize=(3,), meta=np.ndarray>
    LnudgeM2CLM     int32 ...
    LnudgeM3CLM     int32 ...
    LnudgeTCLM      (tracer) int32 dask.array<chunksize=(3,), meta=np.ndarray>
    spherical       int32 ...
    xl              float64 ...
    el              float64 ...
    Vtransform      int32 ...
    Vstretching     int32 ...
    theta_s         float64 ...
    theta_b         float64 ...
    Tcline          float64 ...
    hc              float64 ...
    grid            int32 ...
    Cs_r            (s_rho) float64 dask.array<chunksize=(30,), meta=np.ndarray>
    Cs_w            (s_w) float64 dask.array<chunksize=(31,), meta=np.ndarray>
 

In [8]:
xislice=slice(370,375)
etaslice=slice(140,145)
tslice = slice(0,len(ds.ocean_time))

dsdz = grid.derivative(ds.salt, 'Z', boundary = 'extend')

#salinity variance dissipation - denoted by chi.
chi = 2*(ds.AKs*(dsdz**2)).isel(ocean_time = tslice, 
                                eta_rho = etaslice, 
                                xi_rho = xislice) 

dV = (ds.dx*ds.dy*ds.dz).isel(ocean_time = tslice, 
                              eta_rho = etaslice, 
                              xi_rho = xislice)
V = dV.sum(dim = ['s_rho', 'eta_rho', 'xi_rho'])

#Interpolate to get chi on the rho points 
chiint = grid.interp(chi, 'Z')
chivint = (chivint*dV).sum(dim = ['s_rho', 'eta_rho', 'xi_rho'])
chivint = chivint.rename('chi')

KeyError: 'Z'